In [ ]:
# Setup: Add project root to path
import sys
import os
from pathlib import Path

# Add parent directory to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path configured")

## 1. Initialize Components

Load the necessary components for prompt testing.

In [ ]:
from src.main import RAGPipeline
from src.qa_pipeline.ollama_llm import OllamaLLM
from src.utils.logger import setup_logger

# Setup logging (console only for cleaner output)
setup_logger("Prompt_Testing", log_dir="../logs", log_to_file=False)

# Initialize pipeline
pipeline = RAGPipeline(config_dir="../config")

# Check database
stats = pipeline.get_database_stats()
print(f"Database contains {stats['total_chunks']} chunks from {stats['num_pdfs']} PDFs")

if stats['total_chunks'] == 0:
    print("\nWarning: No chunks in database. Process a PDF first using pipeline_demo.ipynb")

## 2. Define Prompt Templates

Create different prompt templates to test.

In [ ]:
# Prompt templates to test
PROMPT_TEMPLATES = {
    "default": """
Context from documents:
{context}

---

Based on the context above, please answer the following question. 
Cite the page numbers when possible.

Question: {question}

Answer:""",
    
    "detailed": """
You are a helpful assistant that answers questions based on provided document excerpts.

Context:
{context}

---

Instructions:
- Answer the question using ONLY the information from the context above
- Include specific page numbers in your citations (e.g., "Page 3")
- If the context doesn't contain enough information, say so
- Be concise but complete

Question: {question}

Answer:""",
    
    "concise": """
Context:
{context}

Question: {question}

Provide a concise answer with page citations:""",
    
    "structured": """
Document Context:
{context}

---

Question: {question}

Please structure your answer as follows:
1. Direct answer (1-2 sentences)
2. Supporting details from the documents
3. Page references

Answer:"""
}

print("Loaded prompt templates:")
for name in PROMPT_TEMPLATES.keys():
    print(f"  - {name}")

## 3. Test a Single Prompt

Test one prompt template with a sample query.

In [ ]:
# Configuration
test_query = "What is the main topic of this document?"
template_name = "default"  # Change to test different templates

stats = pipeline.get_database_stats()
if stats['total_chunks'] == 0:
    print("No chunks in database. Process a PDF first.")
else:
    print(f"Testing prompt template: {template_name}")
    print(f"Query: {test_query}\n")
    
    # Retrieve context
    retriever_result = pipeline.retriever.retrieve_and_format(test_query)
    context = retriever_result['formatted_context']
    
    # Build prompt
    prompt_template = PROMPT_TEMPLATES[template_name]
    prompt = prompt_template.format(context=context, question=test_query)
    
    print("Generated Prompt:")
    print("=" * 60)
    print(prompt)
    print("=" * 60)
    
    # Generate answer
    answer = pipeline.llm.generate(prompt)
    
    print("\nGenerated Answer:")
    print("=" * 60)
    print(answer)
    print("=" * 60)

## 4. Compare Multiple Prompts

Run the same query through different prompt templates and compare results.

In [ ]:
import time

test_query = "What are the key findings?"

stats = pipeline.get_database_stats()
if stats['total_chunks'] == 0:
    print("No chunks in database. Process a PDF first.")
else:
    print(f"Query: {test_query}\n")
    
    # Retrieve context once (same for all templates)
    retriever_result = pipeline.retriever.retrieve_and_format(test_query)
    context = retriever_result['formatted_context']
    
    results = {}
    
    # Test each template
    for name, template in PROMPT_TEMPLATES.items():
        print(f"Testing template: {name}")
        
        # Build prompt
        prompt = template.format(context=context, question=test_query)
        
        # Generate answer with timing
        start_time = time.time()
        answer = pipeline.llm.generate(prompt)
        elapsed = time.time() - start_time
        
        results[name] = {
            "answer": answer,
            "time": elapsed,
            "length": len(answer),
            "has_page_ref": "page" in answer.lower()
        }
    
    # Display comparison
    print("\n" + "=" * 80)
    print("COMPARISON RESULTS")
    print("=" * 80)
    
    for name, result in results.items():
        print(f"\n[{name.upper()}]")
        print(f"Time: {result['time']:.2f}s")
        print(f"Length: {result['length']} chars")
        print(f"Has page reference: {result['has_page_ref']}")
        print(f"Answer: {result['answer'][:200]}...")
        print("-" * 80)

## 5. Citation Analysis

Check if answers correctly cite page numbers from the context.

In [ ]:
import re

def extract_page_numbers(text):
    """Extract page numbers mentioned in text."""
    # Match patterns like "Page 3", "page 5", "p. 7"
    pattern = r'[Pp]age\s+(\d+)|[Pp]\.\s*(\d+)'
    matches = re.findall(pattern, text)
    # Flatten and filter empty strings
    pages = [int(p) for match in matches for p in match if p]
    return sorted(set(pages))

test_query = "What is discussed in this document?"

stats = pipeline.get_database_stats()
if stats['total_chunks'] == 0:
    print("No chunks in database. Process a PDF first.")
else:
    # Get retrieval result
    retriever_result = pipeline.retriever.retrieve_and_format(test_query)
    
    # Extract actual page numbers from retrieved chunks
    actual_pages = [chunk['page_number'] for chunk in retriever_result['chunks']]
    actual_pages = sorted(set(actual_pages))
    
    print(f"Query: {test_query}")
    print(f"Retrieved chunks from pages: {actual_pages}\n")
    
    # Test with default prompt
    result = pipeline.ask_question(test_query)
    answer = result['answer']
    
    # Extract cited pages
    cited_pages = extract_page_numbers(answer)
    
    print("Answer:")
    print(answer)
    print("\n" + "-" * 60)
    
    print(f"\nCitation Analysis:")
    print(f"Pages in retrieved context: {actual_pages}")
    print(f"Pages cited in answer: {cited_pages}")
    
    # Check accuracy
    correct_citations = [p for p in cited_pages if p in actual_pages]
    incorrect_citations = [p for p in cited_pages if p not in actual_pages]
    
    print(f"\nCorrect citations: {correct_citations}")
    print(f"Incorrect citations: {incorrect_citations}")
    
    if cited_pages:
        accuracy = len(correct_citations) / len(cited_pages) * 100
        print(f"Citation accuracy: {accuracy:.1f}%")
    else:
        print("No citations found in answer")

## 6. Prompt Length Analysis

Analyze how prompt length affects answer quality and generation time.

In [ ]:
import time

test_query = "Summarize the main points"

stats = pipeline.get_database_stats()
if stats['total_chunks'] == 0:
    print("No chunks in database. Process a PDF first.")
else:
    # Test with different top_k values (affects context length)
    top_k_values = [3, 5, 8]
    
    print(f"Query: {test_query}\n")
    print("Testing different context sizes:\n")
    
    for k in top_k_values:
        # Retrieve with different top_k
        pipeline.retriever.top_k = k
        retriever_result = pipeline.retriever.retrieve_and_format(test_query)
        context = retriever_result['formatted_context']
        
        # Build prompt
        prompt = PROMPT_TEMPLATES['default'].format(context=context, question=test_query)
        
        # Measure generation time
        start_time = time.time()
        answer = pipeline.llm.generate(prompt)
        elapsed = time.time() - start_time
        
        # Calculate sizes
        prompt_tokens = len(prompt.split())  # Rough estimate
        answer_tokens = len(answer.split())
        
        print(f"[Top-K = {k}]")
        print(f"Prompt size: ~{prompt_tokens} tokens")
        print(f"Answer size: ~{answer_tokens} tokens")
        print(f"Generation time: {elapsed:.2f}s")
        print(f"Answer preview: {answer[:150]}...")
        print("-" * 60)
    
    # Reset to default
    pipeline.retriever.top_k = pipeline.settings['top_k']

## 7. Custom Prompt Testing

Test your own custom prompt template.

In [ ]:
# Define your custom prompt here
CUSTOM_PROMPT = """
You are analyzing a document. Here is relevant information:

{context}

---

Question: {question}

Please provide:
- A direct answer
- Key supporting points
- Page citations in [Page X] format

Answer:
"""

test_query = "What are the recommendations?"

stats = pipeline.get_database_stats()
if stats['total_chunks'] == 0:
    print("No chunks in database. Process a PDF first.")
else:
    print(f"Query: {test_query}\n")
    
    # Retrieve context
    retriever_result = pipeline.retriever.retrieve_and_format(test_query)
    context = retriever_result['formatted_context']
    
    # Build custom prompt
    prompt = CUSTOM_PROMPT.format(context=context, question=test_query)
    
    # Generate answer
    answer = pipeline.llm.generate(prompt)
    
    print("Answer with custom prompt:")
    print("=" * 60)
    print(answer)
    print("=" * 60)
    
    # Analyze
    cited_pages = extract_page_numbers(answer)
    print(f"\nPages cited: {cited_pages}")
    print(f"Answer length: {len(answer)} characters")

## Summary

This notebook demonstrated:
- Testing multiple prompt templates
- Comparing answer quality and generation time
- Analyzing citation accuracy
- Measuring impact of context length
- Custom prompt experimentation

**Key Findings**:
- Different prompts can significantly affect answer structure and citation quality
- Longer context (higher top_k) may improve answer completeness but increases latency
- Explicit instructions for citations improve page reference accuracy

**Recommendations**:
1. Use structured prompts for better answer organization
2. Explicitly request page citations in the prompt
3. Balance context length vs. generation speed based on use case
4. Test prompts with various query types (factual, conceptual, comparison)